In [70]:
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.api as sm
import statsmodels.formula.api as smf 

rng = np.random.RandomState(0)

In [71]:
# Initialisierung von DataFrames und Data Cleaning
df = pd.read_csv('Laptop-Preise.csv', sep=';', decimal=',')
df = df[df.extern_Schnittstellen != 2300] # Ausreißer löschen
df = df.reset_index(drop=True) # Reset Index
df.drop(['Betriebssystem_OHNE', 'Betriebssystem_Mac', 'Marke_Dell'], axis=1) # Mac und Marke_Apple sind identische Merkmale

df_noPrice = df.drop('Preis', axis=1)
imputed_simul_knn = pd.DataFrame(columns=['Mean KNN_1', 'MSE KNN_1', 'SE KNN_1', 'Alpha KNN_1', 'Mean KNN_3', 'MSE KNN_3', 'SE KNN_3', 'Alpha KNN_3', 'Mean KNN_5', 'MSE KNN_5', 'SE KNN_5', 'Alpha KNN_5'])
imputed_stats_knn = pd.DataFrame(columns=['Mean KNN_1', 'MSE KNN_1', 'SE KNN_1', 'Alpha KNN_1', 'Mean KNN_3', 'MSE KNN_3', 'SE KNN_3', 'Alpha KNN_3', 'Mean KNN_5', 'MSE KNN_5', 'SE KNN_5', 'Alpha KNN_5'], index = np.arange(0.1, 1, 0.1))

imputed_simul_ols = pd.DataFrame(columns=['Mean', 'MSE OLS', 'SE OLS', 'Alpha OLS'])
imputed_stats_ols = pd.DataFrame(columns=['Mean', 'MSE OLS', 'SE OLS', 'Alpha OLS'], index = np.arange(0.1, 1, 0.1))

no_imputation = pd.DataFrame(columns = ['SE', 'Alpha'])


# Skalierung (Standardisierung) von df_noPrice
col_names = df_noPrice.columns
scaler = StandardScaler().fit(df_noPrice.values)
df_noPrice = scaler.transform(df_noPrice.values)
df_noPrice = pd.DataFrame(df_noPrice, columns=col_names)


# Skalierung (Standardisierung) von df mit Preis (Preis ist unverändert)
df_std = df_noPrice.copy()
df_std.insert(0, 'Preis', df['Preis'])

# Wahrer Mittelwert:
true_mean = df['Preis'].mean()

In [72]:
def contains_mean(values):
    return 1 if (true_mean > (np.mean(values) - 1.95996 * stats.sem(values))) and (true_mean < (np.mean(values) + 1.95996 * stats.sem(values))) else 0

In [73]:
# OLS Model 
# fitting the model 
model = smf.ols(formula='Preis ~ '+' + '.join(df_noPrice.columns), data = df_std).fit() 
print(model.summary()) 

# OLS Model 
# fitting the model 
model = sm.OLS(df['Preis'], sm.add_constant(df_noPrice)).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Preis   R-squared:                       0.871
Model:                            OLS   Adj. R-squared:                  0.867
Method:                 Least Squares   F-statistic:                     234.5
Date:                Sun, 14 Jan 2024   Prob (F-statistic):               0.00
Time:                        20:55:30   Log-Likelihood:                -7279.4
No. Observations:                1038   AIC:                         1.462e+04
Df Residuals:                    1008   BIC:                         1.477e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

In [74]:
train_values, test_values, train_labels, test_labels = train_test_split(df_noPrice, df['Preis'], test_size=0.01)

def del_ran(df_exog, labels, chance):
    rand_array = np.random.rand(df_exog.shape[0])
    delete_entries = rand_array < chance
    keep_entries = rand_array >= chance
    
    return [df_exog[delete_entries], labels[delete_entries], df_exog[keep_entries], labels[keep_entries]]

In [75]:
def impute_ols(test_values, test_labels, train_values, train_labels):
    
    # Um zu vermeiden, dass die Matrix am Ende nicht mehr den vollen Rang besitzt, fügen wir einen minimalen Jitter hinzu.
    # So kann die Regression wie gehabt durchgeführt werden. Die Jitter sollte die Werte nicht nennenswert beeinflussen,
    # Aber die Matrix wird auf alle Fälle ihren vollen Range behalten.
    random_values = np.random.uniform(low=-0.001, high=0.001, size=train_values.shape)

    # Add the random values to the original DataFrame
    train_values = train_values + random_values

    # OLS Model
    # fitting the model 
    model = sm.OLS(train_labels, sm.add_constant(train_values)).fit() 

    imputed_values = model.predict(exog = sm.add_constant(test_values, has_constant='add')).tolist()
    
    return [np.mean(list(train_labels) + imputed_values), np.mean((imputed_values-test_labels)**2), stats.sem(list(train_labels) + imputed_values), contains_mean(list(train_labels) + imputed_values)]

In [76]:
def impute_knn(test_values, test_labels, train_values, train_labels):
    
    tree = KDTree(train_values.values, leaf_size=5)

    imputed_values_knn_1 = []
    imputed_values_knn_3 = []
    imputed_values_knn_5 = []

    for index, entry in enumerate(test_values.values):
 
        dist, ind = tree.query([entry], k=5)
        ind = ind[0]

        current_impute_knn_1 = np.mean(train_labels.values[ind][0])
        current_impute_knn_3 = np.mean(train_labels.values[ind][:3])
        current_impute_knn_5 = np.mean(train_labels.values[ind])

        imputed_values_knn_1.append(current_impute_knn_1)
        imputed_values_knn_3.append(current_impute_knn_3)
        imputed_values_knn_5.append(current_impute_knn_5)

    mean_knn_1 = np.mean(list(train_labels.values)+imputed_values_knn_1)
    mean_knn_3 = np.mean(list(train_labels.values)+imputed_values_knn_3)
    mean_knn_5 = np.mean(list(train_labels.values)+imputed_values_knn_5)

    mse_knn_1 = np.mean((test_labels.values - imputed_values_knn_1)**2)
    mse_knn_3 = np.mean((test_labels.values - imputed_values_knn_3)**2)
    mse_knn_5 = np.mean((test_labels.values - imputed_values_knn_5)**2)

    sem_knn_1 = stats.sem(list(train_labels.values)+imputed_values_knn_1)
    sem_knn_3 = stats.sem(list(train_labels.values)+imputed_values_knn_3)
    sem_knn_5 = stats.sem(list(train_labels.values)+imputed_values_knn_5)

    alpha_knn_1 = contains_mean(list(train_labels.values)+imputed_values_knn_1)
    alpha_knn_3 = contains_mean(list(train_labels.values)+imputed_values_knn_3)
    alpha_knn_5 = contains_mean(list(train_labels.values)+imputed_values_knn_5)


    return [mean_knn_1, mse_knn_1, sem_knn_1, alpha_knn_1, mean_knn_3, mse_knn_3, sem_knn_3, alpha_knn_3, mean_knn_5, mse_knn_5, sem_knn_5, alpha_knn_5]

In [77]:
def simul_knn():
    for c in np.arange(0.1, 1, 0.1):

        imputed_simul_knn = pd.DataFrame(columns=['Mean KNN_1', 'MSE KNN_1', 'SE KNN_1', 'Alpha KNN_1', 'Mean KNN_3', 'MSE KNN_3', 'SE KNN_3', 'Alpha KNN_3', 'Mean KNN_5', 'MSE KNN_5', 'SE KNN_5', 'Alpha KNN_5'])

        for i in tqdm(range(1000)):
            temp = del_ran(df_exog = df_noPrice, labels = df['Preis'], chance = c)
            imputed_simul_knn.at[i] = impute_knn(temp[0], temp[1], temp[2], temp[3])

        return_values = []
        
        for column in imputed_simul_knn.columns:
            column_mean = np.mean(imputed_simul_knn[column].to_list())
            return_values.append(column_mean)
        # print(return_values)

        imputed_stats_knn.loc[c] =  return_values
    imputed_stats_knn

simul_knn()
imputed_stats_knn

100%|██████████| 1000/1000 [02:03<00:00,  8.10it/s]


,Mean KNN_1,MSE KNN_1,SE KNN_1,Alpha KNN_1,Mean KNN_3,MSE KNN_3,SE KNN_3,Alpha KNN_3,Mean KNN_5,MSE KNN_5,SE KNN_5,Alpha KNN_5
0.1,1332.225146,93000.318184,23.176042,1.0,1330.860843,75788.146309,23.040802,1.0,1329.888043,82092.808096,22.970502,1.0
0.2,1330.049617,97540.557379,23.111565,1.0,1327.262083,81270.16061,22.817727,1.0,1324.748259,87777.843404,22.67622,1.0
0.3,1327.436285,101860.235095,23.039654,1.0,1322.968322,85701.777929,22.56389,1.0,1318.723743,92076.030473,22.353469,1.0
0.4,1324.309879,109043.39401,22.956428,1.0,1317.514966,92209.775373,22.286294,1.0,1311.585506,97212.663711,22.013145,0.996
0.5,1319.984813,117758.131872,22.833758,0.999,1310.799755,99904.592763,21.961487,0.97,1304.018004,103766.013075,21.629203,0.897
0.6,1314.084437,128793.153066,22.677929,0.974,1301.282393,108958.224322,21.566978,0.799,1295.122319,111172.716945,21.163906,0.593
0.7,1306.72942,143510.141894,22.479979,0.841,1292.140436,119744.728601,21.129251,0.497,1287.280996,120267.769611,20.61696,0.328
0.8,1295.241956,167176.563269,22.20874,0.585,1283.089966,136440.873664,20.534508,0.32,1280.196007,138618.552983,19.782358,0.269
0.9,1280.102154,213691.981141,21.857741,0.369,1269.195635,176258.39798,19.234579,0.238,1261.524387,184392.359491,17.685651,0.178


In [78]:
def simul_ols():
    
    for c in np.arange(0.1, 1, 0.1):
        
        imputed_simul_ols = pd.DataFrame(columns=['Mean', 'MSE OLS', 'SE OLS', 'Alpha OLS'])

        for i in tqdm(range(1000)):
            temp = del_ran(df_exog = df_noPrice, labels = df['Preis'], chance = c)
            imputed_simul_ols.at[i] = impute_ols(temp[0], temp[1], temp[2], temp[3])

        return_values = []

        for column in imputed_simul_ols.columns:
            column_mean = np.mean(imputed_simul_ols[column].to_list())
            return_values.append(column_mean)
        # print(return_values)

        imputed_stats_ols.loc[c] =  return_values
    imputed_stats_ols

simul_ols()

100%|██████████| 1000/1000 [00:19<00:00, 51.49it/s]


Bei der Simulation wird irgendwann ein Fehler geworfen. Das liegt daran, dass, wenn zu viele Einträge gelöscht werden, die Matrix ihren vollen Rang verliert. Einige Dummy-Variablen sind relativ selten. Wenn nun alle Einträge mit der Dummy-Variable auf 1 (oder 0) gelöscht werden, dann wird wegen der linearen Abhängigkeit zur Konstante diese Variable gelöscht - so vermeiden wir Kollinarität.
Wenn dann aber nun im Test-Datensatz die Variable in ihrer Ausprägung 0 (oder 1) besitzt, dann können haben wir dafür keinen passenden Regressionskoeffizienten. Es werden dann 33 Variablen als Input geliefert, obwohl wir nur 32 Regressionskoeffizienten haben. Anschließend wird eine Fehlermeldung geworfen, dass der Eintrag, den wir vorhersagen wollen die falsche shape hat (32 statt 33). 
Es stellt sich zudem an diesem Punkt außerdem die Frage, wie sinnvoll es ist Variablen imputieren zu wollen, bei denen 70% der Einträge fehlen.

Bei den Auswertungen ist zu beachten, dass der wahre Standardfehler von Preis bei 23.2318 liegt.
Wir können also deutlich erkennen, dass jede Imputation der fehlenden Werte den Standardfehler künstlich verringert.

Es ist auch relativ deutlich klar, dass ein höherer k-Wert für KNN dazu führt, dass der Standardfehler weiter sinkt. Das liegt daran, dass bei einem höheren k-Wert der Mittelwert von einem größeren Teil des Datensatzes genommen wird. Dadurch wird der Mittelwert (oder zumindest Mittelwert-nahe Werte) imputiert.

Um zu vermeiden, dass die OLS-Regression einen Fehler wirft, wenn die Matrix nicht den vollen Rang besitzt, haben wir einen Jitter hinzugefügt. Bei jeder Auspräogung wird eine Zufallsvariable von -0.001 bis 0.001 hinzugefügt. Das sollte das Problem beheben, ohne zu viel Bias zu der Simulation hinzuzufügen.

Es treten jedoch Probleme auf, wenn wirklich große Teile des Datensatzes gelöscht werden. Dadurch fällt die Regression (wahrscheinlich) mehr auf den Jitter als erklärende Variable zurück und verzerrt alles unglaublich.

In [80]:
pd.concat([imputed_stats_ols, imputed_stats_knn], axis=1).to_csv('MCAR_Simulation_Normal')
pd.concat([imputed_stats_ols, imputed_stats_knn], axis=1)

,Mean,MSE OLS,SE OLS,Alpha OLS,Mean KNN_1,MSE KNN_1,SE KNN_1,Alpha KNN_1,Mean KNN_3,MSE KNN_3,SE KNN_3,Alpha KNN_3,Mean KNN_5,MSE KNN_5,SE KNN_5,Alpha KNN_5
0.1,1333.754935,77169.057555,23.083019,1.0,1332.225146,93000.318184,23.176042,1.0,1330.860843,75788.146309,23.040802,1.0,1329.888043,82092.808096,22.970502,1.0
0.2,1334.13921,77055.878815,22.929464,1.0,1330.049617,97540.557379,23.111565,1.0,1327.262083,81270.16061,22.817727,1.0,1324.748259,87777.843404,22.67622,1.0
0.3,1333.804103,77999.174104,22.779404,1.0,1327.436285,101860.235095,23.039654,1.0,1322.968322,85701.777929,22.56389,1.0,1318.723743,92076.030473,22.353469,1.0
0.4,1333.69721,78528.253671,22.626562,1.0,1324.309879,109043.39401,22.956428,1.0,1317.514966,92209.775373,22.286294,1.0,1311.585506,97212.663711,22.013145,0.996
0.5,1336.032595,8156544.910055,25.296166,0.997,1319.984813,117758.131872,22.833758,0.999,1310.799755,99904.592763,21.961487,0.97,1304.018004,103766.013075,21.629203,0.897
0.6,1332.845711,3907237.48627,25.123007,0.995,1314.084437,128793.153066,22.677929,0.974,1301.282393,108958.224322,21.566978,0.799,1295.122319,111172.716945,21.163906,0.593
0.7,1336.828899,49302496.185466,45.682132,0.968,1306.72942,143510.141894,22.479979,0.841,1292.140436,119744.728601,21.129251,0.497,1287.280996,120267.769611,20.61696,0.328
0.8,1253.416188,174253536.46611,116.324048,0.872,1295.241956,167176.563269,22.20874,0.585,1283.089966,136440.873664,20.534508,0.32,1280.196007,138618.552983,19.782358,0.269
0.9,1291.827063,2047762195.619254,697.069158,0.492,1280.102154,213691.981141,21.857741,0.369,1269.195635,176258.39798,19.234579,0.238,1261.524387,184392.359491,17.685651,0.178
